In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
base=automap_base()
# reflect the tables
base.prepare(autoload_with=engine)

In [ ]:
# View all of the classes that automap found
base.classes.keys()

In [ ]:
# Save references to each table
measurement=base.classes.measurement
station=base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session=Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# understand the data structure
for column in measurement.__table__.columns:
    print(column)
for column in station.__table__.columns:
    print(column)
# Find the most recent date in the data set.
session.query(func.max(measurement.date)).first()

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
most_recent=session.query(func.max(measurement.date)).first()[0]
year=int(most_recent[0:4])
month=int(most_recent[5:7])
day=int(most_recent[8:])
# Calculate the date one year from the last date in data set.
year_ago = dt.date(year,month,day) - dt.timedelta(days=365)
# Perform a query to retrieve the data and precipitation scores
past_year_data=session.query(measurement.prcp, measurement.date).filter(measurement.date >= year_ago).all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names
df=pd.DataFrame(past_year_data,columns=['percipitation','date'])

# Sort the dataframe by date
df=df.sort_values('date',ascending=True)

# Use Pandas Plotting with Matplotlib to plot the data
df.plot(x='date',y='percipitation',rot=60)
plt.xlabel('date')
plt.ylabel('percipitation in inches')
plt.title('Percipitation over 1 year')

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
session.query(station.id).count()

In [102]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
session.query(measurement.station,func.count(measurement.station)).group_by(measurement.station).order_by(func.count(measurement.station).desc()).all()

[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
session.query(func.min(measurement.tobs),func.max(measurement.tobs),func.avg(measurement.tobs)).where(measurement.station =='USC00519281').all()

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
data=session.query(measurement.tobs).filter(measurement.date >= year_ago).where(measurement.station =='USC00519281').all()
df=pd.DataFrame(data)
df.hist(bins=12)
plt.xlabel('°F')
plt.ylabel('Frequency')
plt.title('Most Frequent Temperatures')

# Close Session

In [ ]:
# Close Session
session.close()